In [1]:
import argparse
import configparser
import os

import numpy as np
import tensorflow as tf
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.models import load_model
from tensorflow.python.keras.utils.np_utils import to_categorical



from model import model as sst_model

train_specInput_root_path = None
train_tempInput_root_path = None
train_label_root_path = None

test_specInput_root_path = None
test_tempInput_root_path = None
test_label_root_path = None

result_path = None
model_save_path = None


input_width = None
specInput_length = None
temInput_length = None

depth_spec = None
depth_tem = None
gr_spec = None
gr_tem = None
nb_dense_block = None
nb_class = None

nbEpoch = None
batch_size = None
lr = None


def read_config(config_path):
    conf = configparser.ConfigParser()
    conf.read(config_path)

    global train_specInput_root_path, train_tempInput_root_path, train_label_root_path, test_specInput_root_path, test_tempInput_root_path, test_label_root_path
    train_specInput_root_path = conf['path']['train_specInput_root_path']
    train_tempInput_root_path = conf['path']['train_tempInput_root_path']
    train_label_root_path = conf['path']['train_label_root_path']
    test_specInput_root_path = conf['path']['test_specInput_root_path']
    test_tempInput_root_path = conf['path']['test_tempInput_root_path']
    test_label_root_path = conf['path']['test_label_root_path']

    global result_path, model_save_path
    result_path = conf['path']['result_path']
    model_save_path = conf['path']['model_save_path']

    if not os.path.exists(result_path):
        os.mkdir(result_path)
    if not os.path.exists(model_save_path):
        os.mkdir(model_save_path)

    global input_width, specInput_length, temInput_length, depth_spec, depth_tem, gr_spec, gr_tem, nb_dense_block, nb_class
    input_width = int(conf['model']['input_width'])
    specInput_length = int(conf['model']['specInput_length'])
    temInput_length = int(conf['model']['temInput_length'])
    depth_spec = int(conf['model']['depth_spec'])
    depth_tem = int(conf['model']['depth_tem'])
    gr_spec = int(conf['model']['gr_spec'])
    gr_tem = int(conf['model']['gr_tem'])
    nb_dense_block = int(conf['model']['nb_dense_block'])
    nb_class = int(conf['model']['nb_class'])

    global nbEpoch, batch_size, lr
    nbEpoch = int(conf['training']['nbEpoch'])
    batch_size = int(conf['training']['batch_size'])
    lr = float(conf['training']['lr'])

read_config('./config/SEED.ini')

Using TensorFlow backend.


In [19]:
all_result_file = open(os.path.join(result_path, 'all_result.txt'), "w")
all_result_file.close()

# Initialize combined arrays
combined_specInput = np.zeros((0, input_width, input_width, specInput_length, 1))
combined_tempInput = np.zeros((0, input_width, input_width, temInput_length, 1))
combined_label = np.zeros((0))

# Loop through subjects and sessions to load and concatenate data
for i in range(1, 15 + 1):
    for j in range(0, 3, 1):
        # combined_specInput = np.concatenate((combined_specInput, np.load(os.path.join(
        #     train_specInput_root_path, f"subject_{i}/section_{j}_data.npy"))), axis=0)
        # combined_tempInput = np.concatenate((combined_tempInput, np.load(os.path.join(
        #     train_tempInput_root_path, f"subject_{i}/section_{j}_data.npy"))), axis=0)
        # combined_label = np.concatenate((combined_label, np.load(os.path.join(
        #     train_label_root_path, f"subject_{i}/section_{j}_label.npy"))), axis=0)

        combined_specInput = np.concatenate((combined_specInput, np.load(os.path.join(
            test_specInput_root_path, f"subject_{i}/section_{j}_data.npy"))), axis=0)
        combined_tempInput = np.concatenate((combined_tempInput, np.load(os.path.join(
            test_tempInput_root_path, f"subject_{i}/section_{j}_data.npy"))), axis=0)
        combined_label = np.concatenate((combined_label, np.load(os.path.join(
            test_label_root_path, f"subject_{i}/section_{j}_label.npy"))), axis=0)

print(combined_specInput.shape)
print(combined_tempInput.shape)
print(combined_label.shape)

# # Shuffle the combined dataset
# index = np.arange(combined_specInput.shape[0])
# np.random.shuffle(index)

# combined_specInput = combined_specInput[index]
# combined_tempInput = combined_tempInput[index]
# combined_label = combined_label[index]

# Convert labels to categorical format
combined_label = [x + 1 for x in combined_label]
combined_label = to_categorical(combined_label, num_classes=3)

model = tf.keras.models.load_model(os.path.join(model_save_path, f'model.h5'))
import time
start_time = time.time()

loss, accuracy = model.evaluate(
    [combined_specInput, combined_tempInput], combined_label)
print(f'time taken: {time.time() - start_time}')
print('\ntest loss', loss)
print('accuracy', accuracy)
        


(90, 32, 32, 5, 1)
(90, 32, 32, 40, 1)
(90,)
3/3 [==============================] - 1s 31ms/step - loss: 1.4089 - accuracy: 0.5556
time taken: 1.1492528915405273

test loss 1.4088603258132935
accuracy 0.5555555820465088


|conditions|loss|acc|time_taken|
|-|-|-|-|
|reduce time to 40s sample, take 40 points |1.414|0.3304|1.6354467868804932|
|reduce time to 50s sample, take 40 points |1.4143825769424438|0.34518519043922424|1.7963125705718994|
|reduce time to 60s sample, take 40 points |1.414535403251648|0.3392592668533325|1.8803508281707764|
|reduce time to 70s sample, take 40 points |1.4145113229751587|0.3466666638851166|1.7162754535675049|
|reduce time to 90s sample, take 40 points |1.414594054222107|0.3377777636051178|1.699704885482788|
|reduce time to 110s sample, take 40 points |1.4146376848220825|0.3318518400192261|1.9813542366027832|